<a href="https://colab.research.google.com/github/josegomes37/Data_ANEEL_GD/blob/main/Solicitacao_ANEEL_gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
from pandas import json_normalize
#import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

def get_GD_data(url):

  #buscando dados da API#######
  #Buscando com várias requisições
  df_aneel = pd.DataFrame()
  #dados = pd.DataFrame()

  offset = 0
  while True:
        url_base = url
        #inserir parametro limit para definir quantidade de linhas acessada
        #a cada requisição à API (Valor máximo de 32000 de acordo com a documentação)
        limit = 32000
        url_x = url_base + '&offset=' + str(offset) + '&limit=' + str(limit)
        #fazendo o request da web page e retornando um objeto json
        response = requests.get(url_x).json()
        r = json.dumps(response)
        #converse a str json para py dict
        loaded_r = json.loads(r)
        #print(loaded_r)
        total_data_rows = loaded_r['result']['total']
        #busca a tabela de dados desejada
        dados = json_normalize(loaded_r['result']['records'])
        #dados = pd.DataFrame(loaded_r['result']['records'])
        df_aneel = pd.concat([df_aneel, dados])
        offset += 32000
        #print(offset, total_data_rows)
        if offset >= total_data_rows:
          #print('estou dentro do if')

          break








  #filtra as colunas
  dados_aneel = df_aneel.loc[:, ['_id', 'NomeTitularEmpreendimento','DatGeracaoConjuntoDados',
                              'SigAgente', 'DscClasseConsumo',
                              'DscClasseConsumo', 'CodSubGrupoTarifario',
                              'DscSubGrupoTarifario', 'SigUF', 'NomRegiao',
                              'NomMunicipio', 'SigTipoConsumidor', 'NumCPFCNPJ',
                              'SigModalidadeEmpreendimento', 'SigTipoGeracao',
                              'DscPorte', 'NumCoordNEmpreendimento', 'NumCoordEEmpreendimento',
                              'MdaPotenciaInstaladaKW','DthAtualizaCadastralEmpreend']]
  #buscas as linhas referentes a geração no RN


  #dados_aneel = dados_aneel.loc[dados_aneel.SigUF == 'RN', :]
  #remove linhas sem a informação da UF
  dados_aneel = dados_aneel.dropna(subset=['SigUF'])
  #filtra dados de apenas usinas fotovoltaicas
  dados_aneel = dados_aneel.loc[dados_aneel.SigTipoGeracao == 'UFV', :]
  #cria novas colunas referentes ao mês e ano (pode ajudar na fase de visualização gráfica)
  #col_data = pd.DatetimeIndex(dados_aneel['DatGeracaoConjuntoDados'])
  col_data = pd.DatetimeIndex(dados_aneel['DthAtualizaCadastralEmpreend'])
  dados_aneel['Ano'] = col_data.year
  dados_aneel['Mes'] = col_data.month_name()
  #converte os valores objeto --> float
  dados_aneel['MdaPotenciaInstaladaKW'] = dados_aneel['MdaPotenciaInstaladaKW'].str.replace(',', '.').astype(float)
  #função para criar filtro de potência (usado para filtrar no powerBI)
  def filter_power(MdaPotenciaInstaladaKW):

    if MdaPotenciaInstaladaKW <= 25:
      return '<= 25 kW'
    elif (MdaPotenciaInstaladaKW > 25) & (MdaPotenciaInstaladaKW <= 110):
      return '> 25 kW e <= 110 kW'
    elif (MdaPotenciaInstaladaKW > 110) & (MdaPotenciaInstaladaKW <= 250):
      return ' > 110 kW e <= 250 kW'
    else:
      return '> 250 kW'

  #função para criar filtro de potência (usado para filtrar no powerBI)
    ## filtro extra
  def filter_power_extra(MdaPotenciaInstaladaKW):

    if MdaPotenciaInstaladaKW <= 110:
      return '<= 110 kW'
    #elif (MdaPotenciaInstaladaKW > 25) & (MdaPotenciaInstaladaKW <= 110):
     # return '> 25 kW e <= 110 kW'
    elif (MdaPotenciaInstaladaKW > 110) & (MdaPotenciaInstaladaKW <= 250):
      return ' > 110 kW e <= 250 kW'
    else:
      return '> 250 kW'

  #cria nova coluna
  dados_aneel['Filter_power'] = dados_aneel['MdaPotenciaInstaladaKW'].apply(filter_power)
  #cria nova coluna
  dados_aneel['Filter_power_extra'] = dados_aneel['MdaPotenciaInstaladaKW'].apply(filter_power_extra)
  #return dados_aneel.to_csv('/content/drive/MyDrive/Data_analsys/Dados_ANEEL.csv', index=False)
  return dados_aneel

#setado url base + parâmetro para filtrar dados do RN
url = 'https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id=b1bd71e7-d0ad-4214-9053-cbd58e9564a7&q=RN'
data_GD = get_GD_data(url)
